In [1]:
import numpy as np
import os
import random
import pandas as pd
import datetime
import math
from math import e
import datetime
from scipy.interpolate import interp1d
import time, datetime
from pandas.core.frame import DataFrame
from db_models import GmsMonitor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
def pd_aggregation(data):
    dense_series_1s = data.resample('1S').interpolate("linear")
    series_1m = dense_series_1s.groupby(pd.Grouper(freq='5Min')).aggregate(np.mean)
    return series_1m

In [3]:
def mongodb_connect():
    import mongoengine
    mongoengine
    mongoengine.connect('nfca_db', host='dgx.server.ustb-ai3d.cn', port=27017, username='nfca', password='nfca')

mongodb_connect()

In [4]:
def queryset2df(query_data):
    """
    queryset转dataframe
    :return:
    """
    dic = {"time":[], "value":[]}
    for q in query_data:
        dic["time"].append(q['time'])
        dic["value"].append(q['Monitoring_value'])
    # 存在数据库多条目同一时间同一值、多条目同一时间不同值的情况，仅保留首条数据
    df = pd.DataFrame(dic).drop_duplicates(subset=['time'])
    return df

In [5]:
df = pd.read_csv('2023_label_1.csv', low_memory=False)

In [6]:
df['time'] = pd.to_datetime(df['time'], format='%Y/%m/%d %H:%M')
df['time'][0]

Timestamp('2022-01-22 23:20:00')

In [7]:
df = df.set_index(['time'], drop=True)
df

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,label
time,,,,,,,,,,,,
2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,1
2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,1
2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,1
2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,1
2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,1
2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,1
2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,1


In [8]:
# df_1 = df.resample('1Min').interpolate("zero")
# df_1 = df_1.drop(df_1.index[0])
df_1 = df


In [9]:
out_f_setting_query = (GmsMonitor.objects.filter(
    time__gte=df_1.index[0],
    time__lt=df_1.index[-1],
    point_id=73
).order_by("time"))
out_f_setting = []
for query in out_f_setting_query:
    out_f_setting.append(query.to_mongo().to_dict())
out_f_setting = queryset2df(out_f_setting).set_index(['time'], drop=False)

# 最后时间的前一个时间
past_set_point =  GmsMonitor.objects.filter(
    time__lt=df_1.index[0],
    point_id=73
).order_by("-time").limit(1)[0].to_mongo().to_dict()
past_time = past_set_point['time']
past_set_value = past_set_point['Monitoring_value']
out_f_setting.loc[past_time] = [past_time, past_set_value]

# 最新时间
last_set_point = GmsMonitor.objects.filter(
    point_id=73
).order_by("-time").limit(1)[0].to_mongo().to_dict()
last_time = datetime.datetime.now()
last_set_value = last_set_point['Monitoring_value']
out_f_setting.loc[last_time] = [last_time, last_set_value]

# 0阶插值\pad上采样
out_f_setting = out_f_setting.sort_index()
out_f_setting = out_f_setting.resample('1S').pad().groupby(pd.Grouper(freq='1Min')).aggregate(np.mean)


out_f_setting = out_f_setting.loc[df_1.index[0]: df_1.index[-1]]   # 和out_c区间保持一致
out_f_setting.rename(columns={"value":"out_f_setting"}, inplace=True)

In [10]:
out_f_setting
df_1 = pd.concat([df_1, out_f_setting],axis=1)

In [11]:
out_c_setting_query = (GmsMonitor.objects.filter(
    time__gte=df_1.index[0],
    time__lt=df_1.index[-1],
    point_id=70
).order_by("time"))
out_c_setting = []
for query in out_c_setting_query:
    out_c_setting.append(query.to_mongo().to_dict())
out_c_setting = queryset2df(out_c_setting).set_index(['time'], drop=False)

# 最后时间的前一个时间
past_set_point =  GmsMonitor.objects.filter(
    time__lt=df_1.index[0],
    point_id=70
).order_by("-time").limit(1)[0].to_mongo().to_dict()
past_time = past_set_point['time']
past_set_value = past_set_point['Monitoring_value']
out_c_setting.loc[past_time] = [past_time, past_set_value]

# 最新时间
last_set_point = GmsMonitor.objects.filter(
    point_id=73
).order_by("-time").limit(1)[0].to_mongo().to_dict()
last_time = datetime.datetime.now()
last_set_value = last_set_point['Monitoring_value']
out_c_setting.loc[last_time] = [last_time, last_set_value]

# 0阶插值\pad上采样
out_c_setting = out_c_setting.sort_index()
out_c_setting = out_c_setting.resample('1S').pad().groupby(pd.Grouper(freq='1Min')).aggregate(np.mean)


out_c_setting = out_c_setting.loc[df_1.index[0]: df_1.index[-1]]   # 和out_c区间保持一致
out_c_setting.rename(columns={"value":"out_c_setting"}, inplace=True)

In [12]:
out_c_setting
df_1 = pd.concat([df_1, out_c_setting],axis=1)

In [13]:
df_1

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,label,out_f_setting,out_c_setting
time,,,,,,,,,,,,,,
2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,1.0,150.0,62.5
2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,1.0,150.0,62.5
2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,1.0,150.0,62.5
2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,1.0,150.0,62.5
2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,1.0,150.0,62.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,1.0,180.0,62.0
2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,1.0,180.0,62.0
2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,1.0,180.0,62.0


In [14]:
df_2 = df_1.dropna(axis=0, how='any')
len(df_2)

56878

In [15]:
# df_2 = df_2.drop(df_2.index[1])

In [16]:
# df_2

In [ ]:
i = 0
while i < len(df_2):
    if i == 0: 
        i = i + 1
    elif i == len(df_2):
        print("Data delete over!")
        break
    elif df_2['pressure'][i] == df_2['pressure'][i-1]:
        df_2 = df_2.drop(df_2.index[i])
        print("Empty time_range", i)
    elif df_2['label'][i] != 1:
        df_2 = df_2.drop(df_2.index[i])
        print("label error", i)
    else:
        i = i + 1

In [19]:
df_2

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,label,out_f_setting,out_c_setting
time,,,,,,,,,,,,,,
2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,1.0,150.0,62.5
2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,1.0,150.0,62.5
2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,1.0,150.0,62.5
2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,1.0,150.0,62.5
2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,1.0,150.0,62.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,1.0,180.0,62.0
2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,1.0,180.0,62.0
2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,1.0,180.0,62.0


In [20]:
df_2.to_csv("df_test_1min.csv", index=True)

In [21]:
df_3 = df_2

In [22]:
df_3.drop(columns = ['label'],inplace = True)

In [23]:
df_3

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting
time,,,,,,,,,,,,,
2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5
2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5
2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5
2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5
2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0
2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0
2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0


In [24]:
reward_1 = - pow((df_3['out_c'] - df_3['out_c_setting']), 2)
reward_2 = -1/(1-(df_3['rake_1'] / 55)) + -1/(1-(df_3['rake_2'] / 55)) + -1/(1-(df_3['rake_3'] / 55)) + -1/(1-(df_3['rake_4'] / 55))
df_3['reward'] = reward_1 + 0.5*reward_2
df_3

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward
time,,,,,,,,,,,,,,
2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119
2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084
2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529
2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288
2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905
2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663
2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472


In [25]:
df_3['error'] = df_3['reward']

In [26]:
df_3['sum_error'] = df_3['error']

In [29]:
df_3['not_done'] = 1

In [31]:
df_3

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
time,,,,,,,,,,,,,,,,,
2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119,-3.890119,-3.890119,1
2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084,-4.483084,-4.483084,1
2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529,-4.110529,-4.110529,1
2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288,-5.080288,-5.080288,1
2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486,-4.310486,-4.310486,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905,-45.509905,-45.509905,1
2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663,-34.615663,-34.615663,1
2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472,-39.253472,-39.253472,1


In [32]:
for i in range(len(df_3) - 1):
    df_3['sum_error'][i+1] = df_3['error'][i+1] + 0.8 * df_3['sum_error'][i]

<ipython-input-32-d533cea76a52>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['sum_error'][i+1] = df_3['error'][i+1] + 0.8 * df_3['sum_error'][i]


In [33]:
df_3

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
time,,,,,,,,,,,,,,,,,
2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119,-3.890119,-3.890119,1
2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084,-4.483084,-7.595180,1
2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529,-4.110529,-10.186672,1
2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288,-5.080288,-13.229626,1
2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486,-4.310486,-14.894187,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905,-45.509905,-inf,1
2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663,-34.615663,-inf,1
2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472,-39.253472,-inf,1


In [34]:
df_3.to_csv("df_thickener1_20230201_1min_temp.csv", index=True)

In [42]:
df_4 = pd.read_csv('df_thickener1_20230201_1min_temp.csv')
df_4

,time,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
0,2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119,-3.890119,-3.890119,1
1,2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084,-4.483084,-7.595180,1
2,2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529,-4.110529,-10.186672,1
3,2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288,-5.080288,-13.229626,1
4,2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486,-4.310486,-14.894187,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44782,2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905,-45.509905,-inf,1
44783,2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663,-34.615663,-inf,1
44784,2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472,-39.253472,-inf,1
44785,2023-01-19 23:12:00,5.72,56.4,0.434,166.0,32.5,0.101,45.0,18.6,20.0,22.4,20.8,180.0,62.0,-34.548861,-34.548861,-inf,1


In [43]:
df_4['sum_error'] = df_4['error']
df_4

,time,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
0,2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119,-3.890119,-3.890119,1
1,2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084,-4.483084,-4.483084,1
2,2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529,-4.110529,-4.110529,1
3,2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288,-5.080288,-5.080288,1
4,2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486,-4.310486,-4.310486,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44782,2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905,-45.509905,-45.509905,1
44783,2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663,-34.615663,-34.615663,1
44784,2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472,-39.253472,-39.253472,1
44785,2023-01-19 23:12:00,5.72,56.4,0.434,166.0,32.5,0.101,45.0,18.6,20.0,22.4,20.8,180.0,62.0,-34.548861,-34.548861,-34.548861,1


In [ ]:
num = 0
last_done = 0
for i in range(len(df_4) - 1):
    if i == (len(df_4)-1):
        break
    # print(i)
    if 300 < (datetime.datetime.strptime(df_4['time'][i+1], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(df_4['time'][i], '%Y-%m-%d %H:%M:%S')).seconds:
        #print("time break")
        df_4['not_done'][i] = 0
        if num < 15:
            print("df_drop")
            df_4 = df_4.drop(labels=range(last_done, i))
            df_4.reset_index(drop=True, inplace=True)
            # print(df)
            last_done = last_done
        else:
            last_done = i 
        num = 0
    else:
        df_4['sum_error'][i+1] = df_4['error'][i+1] + 0.7 * df_4['sum_error'][i]
        if df_4['sum_error'][i+1] > 50:
            df_4['sum_error'][i+1] = 50
        elif df_4['sum_error'][i+1] < -50:
            df_4['sum_error'][i+1] = -50
        num = num + 1
    

In [48]:
df_4

,time,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
0,2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119,-3.890119,-3.890119,1
1,2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084,-4.483084,-7.206168,1
2,2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529,-4.110529,-9.154846,1
3,2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288,-5.080288,-11.488680,1
4,2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486,-4.310486,-12.352562,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43642,2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905,-45.509905,-50.000000,1
43643,2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663,-34.615663,-50.000000,1
43644,2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472,-39.253472,-50.000000,1
43645,2023-01-19 23:12:00,5.72,56.4,0.434,166.0,32.5,0.101,45.0,18.6,20.0,22.4,20.8,180.0,62.0,-34.548861,-34.548861,-50.000000,1


In [55]:
df_4.to_csv("df_thickener1_20230201_1min.csv", index=False)

In [56]:
df_5 = pd.read_csv('df_thickener1_20230201_1min.csv')
df_5

,time,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
0,2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119,-3.890119,-3.890119,1
1,2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084,-4.483084,-7.206168,1
2,2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529,-4.110529,-9.154846,1
3,2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288,-5.080288,-11.488680,1
4,2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486,-4.310486,-12.352562,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43642,2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905,-45.509905,-50.000000,1
43643,2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663,-34.615663,-50.000000,1
43644,2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472,-39.253472,-50.000000,1
43645,2023-01-19 23:12:00,5.72,56.4,0.434,166.0,32.5,0.101,45.0,18.6,20.0,22.4,20.8,180.0,62.0,-34.548861,-34.548861,-50.000000,1


In [57]:
df_5['time'] = pd.to_datetime(df_5['time'], format='%Y/%m/%d %H:%M:%S')
df_5 = df_5.set_index(['time'], drop=True)
df_5

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
time,,,,,,,,,,,,,,,,,
2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119,-3.890119,-3.890119,1
2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084,-4.483084,-7.206168,1
2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529,-4.110529,-9.154846,1
2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288,-5.080288,-11.488680,1
2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486,-4.310486,-12.352562,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905,-45.509905,-50.000000,1
2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663,-34.615663,-50.000000,1
2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472,-39.253472,-50.000000,1


In [59]:
df_6 = df_5.resample('5Min').interpolate("zero")
df_6 = df_6.drop(df_6.index[0])
df_6

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
time,,,,,,,,,,,,,,,,,
2022-01-22 23:25:00,37.30,63.0,769.000,174.0,49.3,0.224,45.0,25.7,25.1,26.0,25.8,150.0,62.5,-3.998356,-3.998356,-12.645149,1.0
2022-01-22 23:30:00,37.30,63.0,769.000,174.0,49.3,0.224,45.0,25.7,25.1,26.0,25.8,150.0,62.5,-3.998356,-3.998356,-12.645149,1.0
2022-01-22 23:35:00,39.90,62.4,721.000,185.0,49.9,0.224,45.0,24.1,25.4,25.0,25.2,150.0,62.5,-3.668507,-3.668507,-12.588330,1.0
2022-01-22 23:40:00,35.10,62.6,774.000,179.0,51.0,0.224,45.0,24.4,24.0,24.4,25.4,150.0,62.5,-3.623536,-3.623536,-12.247671,1.0
2022-01-22 23:45:00,35.10,62.0,776.000,199.0,50.3,0.224,45.0,24.7,24.6,24.9,24.7,150.0,62.5,-3.883408,-3.883408,-14.264890,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 22:50:00,8.98,55.4,0.434,179.0,35.6,0.109,45.0,19.7,20.3,23.3,21.0,180.0,62.0,-46.807875,-46.807875,-50.000000,1.0
2023-01-19 22:55:00,6.97,56.7,0.434,176.0,33.5,0.109,45.0,19.0,19.7,23.6,19.6,180.0,62.0,-31.285558,-31.285558,-50.000000,1.0
2023-01-19 23:00:00,7.13,57.2,0.434,188.0,31.1,0.109,45.0,20.3,20.2,23.7,20.7,180.0,62.0,-26.303081,-26.303081,-50.000000,1.0


In [ ]:
i = 0
while i < len(df_6):
    if i == 0: 
        i = i + 1
    elif i == len(df_6):
        print("Data delete over!")
        break
    elif df_6['pressure'][i] == df_6['pressure'][i-1]:
        df_6 = df_6.drop(df_6.index[i])
        print("Empty time_range", i)
    else:
        i = i + 1

In [61]:
df_6

,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
time,,,,,,,,,,,,,,,,,
2022-01-22 23:25:00,37.30,63.0,769.000,174.0,49.3,0.224,45.0,25.7,25.1,26.0,25.8,150.0,62.5,-3.998356,-3.998356,-12.645149,1.0
2022-01-22 23:35:00,39.90,62.4,721.000,185.0,49.9,0.224,45.0,24.1,25.4,25.0,25.2,150.0,62.5,-3.668507,-3.668507,-12.588330,1.0
2022-01-22 23:40:00,35.10,62.6,774.000,179.0,51.0,0.224,45.0,24.4,24.0,24.4,25.4,150.0,62.5,-3.623536,-3.623536,-12.247671,1.0
2022-01-22 23:45:00,35.10,62.0,776.000,199.0,50.3,0.224,45.0,24.7,24.6,24.9,24.7,150.0,62.5,-3.883408,-3.883408,-14.264890,1.0
2022-01-22 23:50:00,33.70,62.9,590.000,175.0,50.7,0.224,45.0,25.3,26.0,25.9,26.1,150.0,62.5,-3.930776,-3.930776,-13.838265,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19 19:10:00,39.80,56.7,638.000,149.0,39.9,0.123,45.0,14.6,14.0,17.4,13.8,180.0,62.0,-30.840283,-30.840283,-50.000000,1.0
2023-01-19 22:50:00,8.98,55.4,0.434,179.0,35.6,0.109,45.0,19.7,20.3,23.3,21.0,180.0,62.0,-46.807875,-46.807875,-50.000000,1.0
2023-01-19 22:55:00,6.97,56.7,0.434,176.0,33.5,0.109,45.0,19.0,19.7,23.6,19.6,180.0,62.0,-31.285558,-31.285558,-50.000000,1.0


In [62]:
df_6.to_csv("df_thickener1_20230201_5min.csv", index=True)